In [194]:
import pandas as pd
import numpy as np
import datetime
import math
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import random
import operator

In [196]:
# list of team names currently in associated leagues (including relegated teams from last season to fill DF), as they are displayed on Understat.

epl_rank = ['Manchester City', 'Arsenal', 'Liverpool', 'Manchester United', 'Newcastle United', 'Tottenham', 'Brighton',
            'Aston Villa', 'Brentford', 'Chelsea', 'Fulham', 'Crystal Palace', 'Wolverhampton Wanderers', 'Everton',
            'West Ham', 'Nottingham Forest', 'Bournemouth', 'Leicester', 'Southampton','Burnley','Luton','Sheffield United',
            'Leeds']
ligue1_rank = ['Ajaccio', 'Marseille', 'Auxerre', 'Lens', 'Monaco', 'Toulouse', 'Nantes', 'Angers', 'Lorient',
               'Strasbourg', 'Nice', 'Lyon', 'Paris Saint Germain', 'Clermont Foot', 'Brest', 'Rennes', 'Reims',
               'Montpellier', 'Troyes', 'Lille','Metz','Le Havre']
bundesliga_rank = ['Borussia Dortmund', 'Mainz 05', 'RasenBallsport Leipzig', 'Schalke 04', 'Union Berlin',
                   'Werder Bremen', 'FC Cologne', 'Bayern Munich', 'Borussia M.Gladbach', 'Augsburg',
                   'Eintracht Frankfurt', 'Freiburg', 'Wolfsburg', 'Hertha Berlin', 'Bochum', 'Bayer Leverkusen',
                   'VfB Stuttgart', 'Hoffenheim','FC Heidenheim','Darmstadt']
serieA_rank = ['Sassuolo', 'Fiorentina', 'Torino', 'Inter', 'Cremonese', 'Salernitana', 'Empoli', 'Lazio', 'Napoli',
               'Sampdoria', 'Atalanta', 'Monza', 'Lecce', 'Bologna', 'AC Milan', 'Verona', 'Roma', 'Spezia', 'Udinese',
               'Juventus','Cagliari','Frosinone','Genoa']
laliga_rank = ['Real Madrid', 'Athletic Club', 'Villarreal', 'Athletico Madrid', 'Celta Vigo', 'Barcelona',
               'Real Valladolid', 'Getafe', 'Elche', 'Cadiz', 'Espanyol', 'Almeria', 'Osasuna', 'Girona', 'Real Betis',
               'Valencia', 'Mallorca', 'Rayo Vallecano', 'Real Sociedad', 'Sevilla','Granada','Alaves','Las Palmas']

In [197]:
fixtures = pd.read_csv(r'fixtures.txt')
# fixtures = fixtures.reset_index()
fixtures
TeamA = []
TeamB = []
for item in fixtures["-- Date"]:
    if "--" not in item:
        items = item.split(" v ")
        TeamA.append(items[0])
        TeamB.append(items[1])
    else:
        continue
print(TeamA)
print(TeamB)

['Manchester United', 'Bournemouth', 'Crystal Palace', 'Wolverhampton Wanderers', 'Arsenal', 'Aston Villa', 'Brighton', 'West Ham', 'Liverpool', 'Chelsea', 'Nice', 'Lorient', 'Lens', 'Paris Saint Germain', 'Le Havre', 'Metz', 'Strasbourg', 'Lille', 'Marseille', 'VfB Stuttgart', 'Borussia M.Gladbach', 'Augsburg', 'Bayern Munich', 'RasenBallsport Leipzig', 'Werder Bremen', 'Bochum', 'Eintracht Frankfurt', 'Bayer Leverkusen', 'Napoli', 'Cagliari', 'Sassuolo', 'Bologna', 'Genoa', 'Lecce', 'AC Milan', 'Juventus', 'Fiorentina', 'Lazio', 'Barcelona', 'Valencia', 'Cadiz', 'Granada', 'Girona', 'Alaves', 'Las Palmas', 'Real Madrid', 'Real Betis', 'Almeria']
['Everton', 'Sheffield United', 'Luton', 'Fulham', 'Brentford', 'Tottenham', 'Nottingham Forest', 'Burnley', 'Manchester City', 'Newcastle United', 'Montpellier', 'Lyon', 'Brest', 'Reims', 'Toulouse', 'Clermont Foot', 'Monaco', 'Rennes', 'Nantes', 'Union Berlin', 'FC Cologne', 'FC Heidenheim', 'Mainz 05', 'Darmstadt', 'Borussia Dortmund', 'Fr

In [198]:
# Machine Learning Approach (Logistic Regression) to predict winning teams
def win_Ml(df,gamesheet_list):

    y_win = df.win
    x_win = df.drop(['Team','Opp','Unnamed: 0','Unnamed: 0.1','G','GC','Blank','CS','0_0','btts_if_g','btts_if_c','ID','date_int','played','date','win','draw','lose','Pts','match_ID','H/A','S','SC','QS','QSC'], axis = 1)
    # print(x_win)
    xtrain_win, xtest_win, ytrain_win, ytest_win = train_test_split(x_win,y_win, train_size = 0.8, random_state = 42)
    model_win = LogisticRegression()
    model_win.fit(xtrain_win,ytrain_win)
    y_preds_win = model_win.predict(xtest_win)
    accuracy_win = accuracy_score(ytest_win, y_preds_win)
    my_params = np.array([gamesheet_list])
    win_lk = model_win.predict(my_params)

    return win_lk

def draw_Ml(df,gamesheet_list):

    y_draw = df.draw
    x_draw = df.drop(['Team','Opp','Unnamed: 0','Unnamed: 0.1','G','GC','Blank','CS','0_0','btts_if_g','btts_if_c','ID','date_int','played','date','win','draw','lose','Pts','match_ID','H/A','S','SC','QS','QSC'], axis = 1)
    # print(x_draw)
    xtrain_draw, xtest_draw, ytrain_draw, ytest_draw = train_test_split(x_draw,y_draw, train_size = 0.8, random_state = 42)
    model_draw = LogisticRegression()
    model_draw.fit(xtrain_draw,ytrain_draw)
    y_preds_draw = model_draw.predict(xtest_draw)
    accuracy_draw = accuracy_score(ytest_draw, y_preds_draw)
    my_params = np.array([gamesheet_list])
    draw_lk = model_draw.predict(my_params)

    return draw_lk

def lose_Ml(df,gamesheet_list):

    y_lose = df.lose
    x_lose = df.drop(['Team','Opp','Unnamed: 0','Unnamed: 0.1','G','GC','Blank','CS','0_0','btts_if_g','btts_if_c','ID','date_int','played','date','win','draw','lose','Pts','match_ID','H/A','S','SC','QS','QSC'], axis = 1)
    xtrain_lose, xtest_lose, ytrain_lose, ytest_lose = train_test_split(x_lose,y_lose, train_size = 0.8, random_state = 42)
    model_lose = LogisticRegression()
    model_lose.fit(xtrain_lose,ytrain_lose)
    y_preds_lose = model_lose.predict(xtest_lose)
    accuracy_lose = accuracy_score(ytest_lose, y_preds_lose)
    my_params = np.array([gamesheet_list])
    lose_lk = model_lose.predict(my_params)

    return model_lose

In [199]:
def score_gen(h,a,hg_av,hgc_av,ag_av,agc_av,ov_h,ov_a):
    # Goal Difference
    h_gd = round(((h['GD'].item() - a['GD'].item())/2),0)
    a_gd = round(((a['GD'].item() - h['GD'].item())/2),0)

    # Score expectance (Poisson)

    h_att_str = hg_av/ov_h
    h_def_str = hgc_av/ov_a
    a_att_str = ag_av/ov_a
    a_def_str = agc_av/ov_h
    h_exp = h_att_str * a_def_str * ov_h
    a_exp = a_att_str * h_def_str * ov_a

    # First Goal
    h_fg = (h['first_goal'].item() + a['first_conc'].item())/2
    a_fg = (a['first_goal'].item() + h['first_conc'].item())/2

    # BTTS given H/A goal
    a_btts_if_fgh = (h['btts_if_g'].item() + a['btts_if_c'].item())/2
    h_btts_if_fga = (a['btts_if_g'].item() + h['btts_if_c'].item())/2

    if h_fg > 0.65:
        if a_btts_if_fgh < 0.5:
            h_s = round(((h['GD'].item() - a['GD'].item())/2),0)
            a_s = 0
        else:
            h_s = poisson_team_goal_prob(h_exp)
            a_s = poisson_team_goal_prob(a_exp)

    elif a_fg > 0.65:
        if h_btts_if_fga < 0.5:
            a_s = round(((a['GD'].item() - h['GD'].item())/2),0)
            h_s = 0
        else:
            h_s = poisson_team_goal_prob(h_exp)
            a_s = poisson_team_goal_prob(a_exp)
    else:
        h_s = poisson_team_goal_prob(h_exp)
        a_s = poisson_team_goal_prob(a_exp)

    return h_s, a_s

In [200]:
def params_gen_ML(h,a,har,aar):
    xGH = ((h['xG'].item() + a['xGC'].item())/2)
    xGA = ((a['xG'].item() + h['xGC'].item())/2)
    BTTS = ((h['BTTS'].item() + a['BTTS'].item())/2)
    fg_h = (h['first_goal'].item() + a['first_conc'].item())/2
    fg_a = (a['first_goal'].item() + h['first_conc'].item())/2
    rank = har
    rankvs = aar
    home_list = [xGH,xGA,BTTS,fg_h,fg_a,rank,rankvs]
    away_list = [xGA,xGH,BTTS,fg_a,fg_h,rankvs,rank]
    return home_list, away_list



In [202]:
def poisson_team_goal_prob(g_s):
    poisson_goals = random.poisson(lam=g_s, size=100000)
    goals0 = 0
    goals1 = 0
    goals2 = 0
    goals3 = 0
    goals4 = 0
    goals5 = 0
    goals5plus = 0

    for i in range(0, 10000):
        if poisson_goals[i] == 0:
            goals0 = goals0+1
            prob0 = goals0/ 10000
        elif poisson_goals[i] == 1:
            goals1 = goals1+1
            prob1 = goals1/ 10000
        elif poisson_goals[i] == 2:
            goals2 = goals2+1
            prob2 = goals2/ 10000
        elif poisson_goals[i] == 3:
            goals3 = goals3+1
            prob3 = goals3/ 10000
        elif poisson_goals[i] == 4:
            goals4 = goals4+1
            prob4 = goals4/ 10000
        elif poisson_goals[i] == 5:
            goals5 = goals5+1
            prob5 = goals5/ 10000
        else:
            goals5plus = goals5plus+1
            prob5plus = goals5plus/ 10000

    dict_g = {"0": prob0, "1": prob1, "2": prob2, "3": prob3, "4": prob4, "5": prob5, "6": prob5plus}
    dict_max = (max(dict_g.items(), key = operator.itemgetter(1)))
    goal_max = int(dict_max[0])

    return goal_max





In [203]:

def rank_decision():
    scoresheet_df = pd.DataFrame()
    scoresheet_dict = {}
    teamlist = []
    h_score = []
    a_score = []

    for i in range(0,len(TeamA)):
        Home_Team = TeamA[i]
        # print(Home_Team)
        Away_Team = TeamB[i]
        # print(Away_Team)

        if Home_Team in epl_rank:
            df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank3_away')
            r1_h_df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name="versus_rank1_home_df")
            r2_h_df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name="versus_rank2_home_df")
            r3_h_df =  pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name="versus_rank3_home_df")
            r1_a_df =  pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name="versus_rank1_away_df")
            r2_a_df =  pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name="versus_rank2_away_df")
            r3_a_df =  pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name="versus_rank3_away_df")

        elif Home_Team in ligue1_rank:
            df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank3_away')
            r1_h_df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank1_home_df")
            r2_h_df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank2_home_df")
            r3_h_df =  pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank3_home_df")
            r1_a_df =  pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank1_away_df")
            r2_a_df =  pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank2_away_df")
            r3_a_df =  pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank3_away_df")

        elif Home_Team in bundesliga_rank:
            df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank3_away')
            r1_h_df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank1_home_df")
            r2_h_df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank2_home_df")
            r3_h_df =  pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank3_home_df")
            r1_a_df =  pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank1_away_df")
            r2_a_df =  pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank2_away_df")
            r3_a_df =  pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank3_away_df")

        elif Home_Team in serieA_rank:
            df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank3_away')
            r1_h_df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank1_home_df")
            r2_h_df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank2_home_df")
            r3_h_df =  pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank3_home_df")
            r1_a_df =  pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank1_away_df")
            r2_a_df =  pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank2_away_df")
            r3_a_df =  pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank3_away_df")

        elif Home_Team in laliga_rank:
            df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank3_away')
            r1_h_df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank1_home_df")
            r2_h_df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank2_home_df")
            r3_h_df =  pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank3_home_df")
            r1_a_df =  pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank1_away_df")
            r2_a_df =  pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank2_away_df")
            r3_a_df =  pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank3_away_df")
            
        else:
            print("This Team doesnt feature in our database!")
            i += 1
        
        # Historic Rank data vs Form-Influenced Rank Data
        hf = l10_table_h.loc[l10_table_h["Team"] == Home_Team]
        hrec = table_h.loc[table_h["Team"] == Home_Team]
        af = l10_table_a.loc[l10_table_a["Team"] == Away_Team]
        arec = table_a.loc[table_a["Team"] == Away_Team]
        hf_rank = float(hf['Rank'].item()) #2
        # print(hf_rank)
        hf_rankvs = float(hf['Rankvs'].item()) #2.4
        # print(hf_rankvs)
        hrec = float(hrec['Rank'].item()) #2
        # print(hrec)
        af_rank = float(af['Rank'].item()) #2
        # print(af_rank)
        af_rankvs = float(af['Rankvs'].item()) #2.25
        # print(af_rankvs)
        arec = float(arec['Rank'].item()) #3
        # print(arec)
        # Home
        if (hf_rank == hrec) or (hf_rank < hrec and hf_rankvs > 2.25) or (hf_rank > hrec and hf_rankvs < 1.75):
            if hf_rank == 1.0:
                home_adj_rank = 1.0
                away = r1_table_a.loc[r1_table_a["Team"] == Away_Team]
                away_df = r1_a_df.loc[r1_a_df["Team"] == Away_Team]
                ov_away_g = r1_a_df['G'].mean()
            elif hf_rank == 2.0:
                home_adj_rank = 2.0
                away = r2_table_a.loc[r2_table_a["Team"] == Away_Team]
                away_df = r2_a_df.loc[r2_a_df["Team"] == Away_Team]
                ov_away_g = r2_a_df['G'].mean()
            else:
                home_adj_rank = 3.0
                away = r3_table_a.loc[r3_table_a["Team"] == Away_Team]
                away_df = r3_a_df.loc[r3_a_df["Team"] == Away_Team]
                ov_away_g = r3_a_df['G'].mean()
        elif ((hf_rank < hrec) and (hf_rankvs <= 2.25)):
            if hf_rank == 1.0:
                home_adj_rank = 1.0
                away = r1_table_a.loc[r1_table_a["Team"] == Away_Team]
                away_df = r1_a_df.loc[r1_a_df["Team"] == Away_Team]
                ov_away_g = r1_a_df['G'].mean()
            else:
                home_adj_rank = 2.0
                away = r2_table_a.loc[r2_table_a["Team"] == Away_Team]
                away_df = r2_a_df.loc[r2_a_df["Team"] == Away_Team]
                ov_away_g = r2_a_df['G'].mean()
        elif ((hf_rank > hrec) and (hf_rankvs >= 1.75)):
            if hf_rank == 2.0:
                home_adj_rank = 2.0
                away = r2_table_a.loc[r2_table_a["Team"] == Away_Team]
                away_df = r2_a_df.loc[r2_a_df["Team"] == Away_Team]
                ov_away_g = r2_a_df['G'].mean()
            else:
                home_adj_rank = 3.0
                away = r3_table_a.loc[r3_table_a["Team"] == Away_Team]
                away_df = r3_a_df.loc[r3_a_df["Team"] == Away_Team]
                ov_away_g = r1_a_df['G'].mean()
        else:
            print("???")
        # Away
        if (af_rank == arec) or (af_rank < arec and af_rankvs > 2.25) or (af_rank > arec and af_rankvs < 1.75):
            if af_rank == 1.0:
                away_adj_rank = 1.0
                home = r1_table_h.loc[r1_table_h["Team"] == Home_Team]
                home_df = r1_h_df.loc[r1_h_df["Team"] == Home_Team]
                ov_home_g = r1_h_df['G'].mean()
            elif af_rank == 2.0:
                away_adj_rank = 2.0
                home = r2_table_h.loc[r2_table_h["Team"] == Home_Team]
                home_df = r2_h_df.loc[r2_h_df["Team"] == Home_Team]
                ov_home_g = r2_h_df['G'].mean()
            else:
                away_adj_rank = 3.0
                home = r3_table_h.loc[r3_table_h["Team"] == Home_Team]
                home_df = r3_h_df.loc[r3_h_df["Team"] == Home_Team]
                ov_home_g = r3_h_df['G'].mean()
        elif ((af_rank < arec) and (af_rankvs <= 2.25)):
            if af_rank == 1.0:
                away_adj_rank = 1.0
                home = r1_table_h.loc[r1_table_h["Team"] == Home_Team]
                home_df = r1_h_df.loc[r1_h_df["Team"] == Home_Team]
                ov_home_g = r1_h_df['G'].mean()
            else:
                away_adj_rank = 2.0
                home = r2_table_h.loc[r2_table_h["Team"] == Home_Team]
                home_df = r2_h_df.loc[r2_h_df["Team"] == Home_Team]
                ov_home_g = r2_h_df['G'].mean()
        elif ((af_rank > arec) and (af_rankvs >= 1.75)):
            if af_rank == 2.0:
                away_adj_rank = 2.0
                home = r2_table_h.loc[r2_table_h["Team"] == Home_Team]
                home_df = r2_h_df.loc[r2_h_df["Team"] == Home_Team]
                ov_home_g = r2_h_df['G'].mean()
            else:
                away_adj_rank = 3.0
                home = r3_table_h.loc[r3_table_h["Team"] == Home_Team]
                home_df = r3_h_df.loc[r3_h_df["Team"] == Home_Team]
                ov_home_g = r3_h_df['G'].mean()
        else:
            print("???")
        av_home_g = home_df['G'].mean()
        av_home_gc = home_df['GC'].mean()
        av_away_g = away_df['G'].mean()
        av_away_gc = away_df['GC'].mean()
        score_h,score_a = score_gen(home,away,av_home_g,av_home_gc,av_away_g,av_away_gc,ov_away_g,ov_home_g)
        teamlist.append(f'{Home_Team} v {Away_Team}')
        h_score.append(score_h)
        a_score.append(score_a)
        # print(home)
        # print(away)
        # home_stat_list,away_stat_list = stat_gen(home,away,home_adj_rank,away_adj_rank)
        # win_sum_h = home_df['win'].sum()
        # print(win_sum_h)
        # win_sum_a = away_df['win'].sum()
        # print(win_sum_a)
        # draw_sum_h = home_df['draw'].sum()
        # print(draw_sum_h)
        # draw_sum_a = away_df['draw'].sum()
        # print(draw_sum_a)
        # lose_sum_h = home_df['lose'].sum()
        # print(lose_sum_h)
        # lose_sum_a = away_df['lose'].sum()
        # print(lose_sum_a)
        # if win_sum_h >=2 and win_sum_a >=2:
        #     win_lk_h = win_ml(home_df,home_stat_list)
        #     win_lk_a = win_ml(away_df,away_stat_list)
        # else:
        #     win_lk_h = "Small sample"
        #     win_lk_a = "Small sample"
        # if draw_sum_h >= 2 and draw_sum_a >= 2:
        #     draw_lk_h = draw_ml(home_df,home_stat_list)
        #     draw_lk_a = draw_ml(away_df,away_stat_list)
        # else:
        #     draw_lk_h = "Small sample"
        #     draw_lk_a = "Small sample"
        # if lose_sum_h >= 2 and lose_sum_a >= 2:
        #     lose_lk_h = lose_ml(home_df,home_stat_list)
        #     lose_lk_a = lose_ml(away_df,away_stat_list)
        # else:
        #     lose_lk_h = "Small sample"
        #     lose_lk_a = "Small sample"

        # print(f"{win_lk_h},{draw_lk_h},{lose_lk_h},{win_lk_a},{draw_lk_a},{lose_lk_a}")
        # print(away_df)
        # score_h, score_a, GH, GA, xGH, xGA, GDH, GDA, FGH, FGA, BTTSIFHG, BTTSIFAG = score_gen(home,away)
        # # print(f"{home['Team'].item()} v {away['Team'].item()}")
        # score_list = (f"{home['Team'].item()} {score_h} - {score_a} {away['Team'].item()}")
        # # scoresheet_list.append(score_list)
        # df_game = df_game.T
        # scoresheet_df = pd.concat([scoresheet_df, df_game])
        # i += 1
    scoresheet_df['Team'] = teamlist
    scoresheet_df['Home Score'] = h_score
    scoresheet_df['Away Score'] = a_score
    return scoresheet_df
scoresheet = rank_decision()
scoresheet

KeyError: 'Rankvs'

In [ ]:
# def poisson_dist():
    

In [339]:
date = datetime.datetime.now().strftime("%m-%d-%Y")

with pd.ExcelWriter("../Frontend/scorepredictions.xlsx") as writer:
    scoresheet.to_excel(writer, sheet_name=f"gamesheet_{date}")